In [ ]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 30.6MB/s 
     |████████████████████████████████| 1.2MB 47.1MB/s 
     |████████████████████████████████| 901kB 46.2MB/s 
     |████████████████████████████████| 3.0MB 45.4MB/s 


In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

import tensorflow as tf
import json

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#### load data and prepare data

In [ ]:
#load data 
df = pd.read_csv("Constraint_Train.csv")

In [ ]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
df['label'].unique()

array(['real', 'fake'], dtype=object)

In [ ]:
df['label'] = df['label'].replace({"fake":0, "real":1})

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df['tweet'], df['label'], 
                                                                    random_state=2018, 
                                                                    test_size=0.4, 
                                                                    stratify=df['label'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
training_sentences = train_text.to_list()
validation_sentences = val_text.to_list()
training_labels = train_labels.to_list()
validation_labels = val_labels.to_list()

In [ ]:
print(len(training_sentences))
print(len(validation_sentences))

3854
1285


In [ ]:
#### Setup BERT and run training

# https://huggingface.co/transformers/pretrained_models.html
# distilbert-base-multilingual-cased

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [ ]:

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                              num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
241/241 [==============================] - 220s 880ms/step - loss: 0.2088 - accuracy: 0.9193 - val_loss: 0.1286 - val_accuracy: 0.9494
Epoch 2/3
241/241 [==============================] - 221s 916ms/step - loss: 0.0511 - accuracy: 0.9826 - val_loss: 0.1241 - val_accuracy: 0.9502
Epoch 3/3
241/241 [==============================] - 221s 916ms/step - loss: 0.0282 - accuracy: 0.9907 - val_loss: 0.1479 - val_accuracy: 0.9611


In [ ]:
model.save_pretrained("/tmp/sentiment_custom_model")

In [ ]:
#### Load saved model and run predict function

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/tmp/sentiment_custom_model")

Some weights of the model checkpoint at /tmp/sentiment_custom_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /tmp/sentiment_custom_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
predicted_labels = []

In [ ]:
test_text = test_text.to_list()

In [ ]:
test_labels = test_labels.to_list()

In [ ]:
#test_sentence = "With their homes in ashes, residents share harrowing tales of survival after massive wildfires kill 15"
for test_sentence in test_text:
    predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

    tf_output = loaded_model.predict(predict_input)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
    prediction = np.argmax([tf_prediction], axis = 1)[0]
    predicted_labels.append(prediction)


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       612
           1       0.97      0.97      0.97       673

    accuracy                           0.97      1285
   macro avg       0.97      0.97      0.97      1285
weighted avg       0.97      0.97      0.97      1285

